# Milan Real Estate Data Analysis

This project involves analyzing real estate data for the city of Milan, focusing on cleaning the data, identifying patterns, and selecting the most relevant data columns for analysis.

In [147]:
import requests
import pandas as pd

# URL of the API
url = "https://www.immobiliare.it/api-next/search-list/real-estates/"

# Headers for the request
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en,en-US;q=0.9,en-GB;q=0.8,it;q=0.7,it-IT;q=0.6',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0',
    'Referer': 'https://www.immobiliare.it/vendita-case/milano/?criterio=prezzo&ordine=desc',
    'Sec-CH-UA': '"Microsoft Edge";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
    'Sec-CH-UA-Mobile': '?0',
    'Sec-CH-UA-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
}

# Initialize an empty list to store all the results
all_results = []
page = 1
max_pages = 10  # You can increase this based on how many pages you want to fetch

while True:
    # Parameters of the request, including page number
    params = {
        'fkRegione': 'lom',
        'idProvincia': 'MI',
        'idComune': '8042',
        'idNazione': 'IT',
        'idContratto': '1',
        'idCategoria': '1',
        'criterio': 'rilevanza',
        '__lang': 'it',
        'pag': str(page),  # Incrementing the page number
        'paramsCount': '2',
        'path': '/vendita-case/milano/'
    }

    # Send GET request to fetch data
    response = requests.get(url, headers=headers, params=params)

    # Check if request is successful
    if response.status_code == 200:
        data = response.json()
        
        # Extract 'results' key from JSON
        results = data.get('results', [])
        
        if not results:
            break  # Stop the loop if no more results are returned

        for item in results:
            real_estate = item.get('realEstate', {})
            properties = real_estate.get('properties', [{}])[0]  # Get first property if available

            # Extracting values from nested dictionaries
            all_results.append({
                'id': real_estate.get('id'),
                'title': real_estate.get('title', ''),
                'price': properties.get('price', {}).get('value', ''),
                'price_range': properties.get('price', {}).get('priceRange', ''),  # Price range if available
                'surface': properties.get('surface', ''),
                'rooms': properties.get('rooms', ''),
                'bathrooms': properties.get('bathrooms', ''),
                'construction_year': properties.get('buildingYear', ''),  # Year of construction
                'floor': properties.get('floor', {}).get('value', ''),  # Floor information
                'energy_class': properties.get('energy', {}).get('class', ''),
                'heating_type': properties.get('energy', {}).get('heatingType', ''),  # Heating type
                'air_conditioning': properties.get('energy', {}).get('airConditioning', ''),  # Air conditioning
                'typology': properties.get('typology', {}).get('name', ''),
                'category': properties.get('category', {}).get('name', ''),  # Property category
                'contract': real_estate.get('contract', ''),
                'description': properties.get('description', ''),
                'condominium_expenses': properties.get('costs', {}).get('condominiumExpenses', ''),  # Monthly expenses
                'city': properties.get('location', {}).get('city', ''),
                'province': properties.get('location', {}).get('province', ''),
                'region': properties.get('location', {}).get('region', ''),
                'macrozone': properties.get('location', {}).get('macrozone', ''),
                'microzone': properties.get('location', {}).get('microzone', ''),
                'latitude': properties.get('location', {}).get('latitude', ''),  # Latitude
                'longitude': properties.get('location', {}).get('longitude', ''),  # Longitude
                'features': properties.get('ga4features', []),  # Additional property features
                'multimedia': properties.get('multimedia', {}),
                'url': properties.get('url', ''),
            })

        # Go to the next page
        page += 1

        # Optional: Stop after a certain number of pages (adjust max_pages as needed)
        if page > max_pages:
            break

    else:
        print(f"Error in request: {response.status_code}")
        break

# Convert to DataFrame
df = pd.DataFrame(all_results)

# Display the DataFrame
display(df.sample(10))
print(df.shape)

,id,title,price,price_range,surface,rooms,bathrooms,construction_year,floor,energy_class,...,city,province,region,macrozone,microzone,latitude,longitude,features,multimedia,url
74,110946749,"Trilocale via Alessandro Astesani 24, Affori, ...",459000,300.001 - 500.000 &euro;,98 m²,3,2,,"1°, con ascensore",,...,Milano,Milano,Lombardia,"Affori, Bovisa",Affori,45.5182,9.1709,"[cancello elettrico, videocitofono, porta blin...","{'photos': [{'id': 1547154239, 'caption': 'Sog...",
168,108929113,Appartamenti di nuova costruzione a Milano,415000,,102 m²,3,,,,,...,Milano,Milano,Lombardia,"Cimiano, Crescenzago, Adriano",Quartiere Adriano,45.5178,9.2500,"[accesso per disabili, parcheggio bici]","{'photos': [{'id': 1511492281, 'caption': 'est...",
25,114763857,"Bilocale via Padova 39, Parco Trotter, Milano",265000,200.001 - 300.000 &euro;,50 m²,2,1,,3°,,...,Milano,Milano,Lombardia,"Pasteur, Rovereto",Parco Trotter,45.4908,9.2228,"[cancello elettrico, fibra ottica, videocitofo...","{'photos': [{'id': 1579375879, 'caption': 'liv...",
149,114779549,"Trilocale via Vitruvio 46, Centrale, Milano",520000,oltre 500.000 &euro;,115 m²,3,1,,"1°, con ascensore",,...,Milano,Milano,Lombardia,"Centrale, Repubblica",Centrale,45.4834,9.2051,"[fibra ottica, videocitofono, porta blindata, ...","{'photos': [{'id': 1579719293, 'caption': 'SOG...",
13,105980495,Appartamenti e Attici di nuova costruzione a M...,427000,,62 m²,1 - 4,,,,,...,Milano,Milano,Lombardia,"Bande Nere, Inganni",Bande Nere,45.4585,9.1322,"[accesso per disabili, parcheggio bici]","{'photos': [{'id': 1402147859, 'caption': 'Com...",
199,112948331,"Attico via Gallarate 55, Certosa, Milano",699000,oltre 500.000 &euro;,203 m²,4,2,,"2 piani: da 7° a 8°, con ascensore",,...,Milano,Milano,Lombardia,"Viale Certosa, Cascina Merlata",Certosa,45.4952,9.1369,"[campo da tennis, cancello elettrico, caminett...","{'photos': [{'id': 1566625009, 'caption': 'gia...",
10,107671673,Appartamenti di nuova costruzione a Milano,428000,,86 m²,3 - 4,,,,,...,Milano,Milano,Lombardia,"Precotto, Turro",Turro,45.4994,9.2288,"[accesso per disabili, parcheggio bici]","{'photos': [{'id': 1447107537, 'caption': 'fac...",
54,109918819,"Appartamento via Patroclo, San Siro, Milano",1400000,oltre 500.000 &euro;,267 m²,5+,3+,,"1°, con ascensore",,...,Milano,Milano,Lombardia,"San Siro, Trenno",San Siro,45.4827,9.1186,"[idromassaggio, impianto di allarme, accesso p...","{'photos': [{'id': 1484109715, 'caption': 'Liv...",
238,111224849,"Bilocale via Alfonso Capecelatro 22, San Siro,...",299000,200.001 - 300.000 &euro;,53 m²,2,1,,"8°, con ascensore",,...,Milano,Milano,Lombardia,"San Siro, Trenno",San Siro,45.4696,9.1323,"[cancello elettrico, fibra ottica, videocitofo...","{'photos': [{'id': 1508913143, 'caption': 'LB'...",
118,112959331,Appartamenti di nuova costruzione a Milano,470000,,59 m²,2 - 4,,,,,...,Milano,Milano,Lombardia,"Porta Vittoria, Lodi",Porta Vittoria,45.4614,9.2270,"[accesso per disabili, parcheggio bici]","{'photos': [{'id': 1542387929, 'caption': '.',...",


(250, 27)


## Step 2: Initial Data Exploration

We will explore the structure of the collected dataset, including the number of rows, columns, and basic descriptive statistics.

In [184]:
# Check the first few rows of the dataset
display(df.head())

# Get the shape of the dataset
print(df.shape)

# Check basic statistics
display(df.describe())

print(df.columns)

,title,price,surface,rooms,bathrooms,typology,macrozone,microzone,latitude,longitude
0,"Bilocale via Carlo Ravizza 40, Amendola - Buon...",430000,64,2,1,Appartamento,"Fiera, Sempione, City Life, Portello",Amendola - Buonarroti,45.4705,9.1488
1,"Bilocale piazza Insubria 10, Martini - Insubri...",310000,50,2,1,Appartamento,"Porta Vittoria, Lodi",Martini - Insubria,45.4532,9.2195
2,"Bilocale via Giovanni Battista Bertini 21, Pao...",620000,63,2,1,Appartamento,"Cenisio, Sarpi, Isola",Paolo Sarpi,45.4829,9.1726
3,"Trilocale via Ugo la Malfa 6, Quartiere Adrian...",359000,100,3,1,Appartamento,"Cimiano, Crescenzago, Adriano",Quartiere Adriano,45.5167,9.2464
4,"Villa unifamiliare via Bolzano 9, Rovereto, Mi...",2000000,400,0,0,Villa unifamiliare,"Pasteur, Rovereto",Rovereto,45.4962,9.2210


(99, 10)


,price,surface,rooms,bathrooms,latitude,longitude
count,9.900000e+01,99.000000,99.000000,99.000000,99.000000,99.000000
mean,6.994960e+05,111.414141,2.878788,1.474747,45.477016,9.188716
std,5.319093e+05,64.795126,0.992551,0.733227,0.023302,0.033415
min,7.500000e+04,25.000000,0.000000,0.000000,45.422600,9.087100
25%,3.620000e+05,65.000000,2.000000,1.000000,45.458000,9.166050
50%,4.700000e+05,96.000000,3.000000,1.000000,45.476700,9.185900
75%,7.950000e+05,140.000000,3.000000,2.000000,45.494950,9.215650
max,3.200000e+06,400.000000,5.000000,3.000000,45.529800,9.253600


Index(['title', 'price', 'surface', 'rooms', 'bathrooms', 'typology',
       'macrozone', 'microzone', 'latitude', 'longitude'],
      dtype='object')


## Step 3: Data Cleaning

Before we can proceed with the analysis, it is essential to clean the data. This involves:
1. **Checking for missing values** and deciding how to handle them.
2. **Ensuring all numeric fields** (e.g., price and surface area) are in the correct format.
3. **Removing duplicates** if any exist.
4. **Handling outliers** or values that seem unreasonable.

In this step, we will review the data and apply cleaning techniques to ensure consistency and quality.


In [136]:
# Checking for missing values in the DataFrame
missing_data = df.isnull().sum()

# Display the columns with missing values
missing_data[missing_data > 0]

Series([], dtype: int64)

### Ensuring Correct Data Types

Next, we need to ensure that our numeric fields like `price` and `surface` are in the correct format and are ready for analysis.


In [183]:
# Convert 'price' and 'surface' to numeric after removing any non-numeric characters
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['surface'] = df['surface'].str.replace('m²', '').str.replace(',', '').str.strip()
df['surface'] = pd.to_numeric(df['surface'], errors='coerce')
df = df[~df['typology'].str.contains('progetto', case=False, na=False)]
df['rooms'] = pd.to_numeric(df['rooms'], errors='coerce').fillna(0).astype(int)
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce').fillna(0).astype(int)
df['construction_year'] = pd.to_numeric(df['construction_year'], errors='coerce').fillna(0).astype(int)

# Recheck the data types after conversion
df.dtypes

KeyError: 'construction_year'

### Removing Duplicates

Finally, we will check for duplicate rows and remove them if necessary.

In [138]:
# Check for columns that contain list or dictionary types
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, (list, dict))).any():
        print(f"Column '{col}' contains lists or dictionaries.")

Column 'features' contains lists or dictionaries.
Column 'multimedia' contains lists or dictionaries.


In [139]:
# List columns with unhashable types (e.g., lists or dictionaries)
columns_to_exclude = ['multimedia', 'features']  # Example based on previous issues

# Check for duplicates excluding those columns
duplicate_rows = df[df.drop(columns=columns_to_exclude).duplicated()]
print(f"Number of duplicate rows: {duplicate_rows.shape[0]}")

# Drop duplicates if needed
df = df.drop_duplicates(subset=[col for col in df.columns if col not in columns_to_exclude])


Number of duplicate rows: 0


## Step 4: Data Selection and Filtering

In this step, we will:
- **Remove unnecessary columns** to keep only the columns relevant for the analysis.
- **Filter the dataset** to include only rows that meet specific criteria (e.g., certain neighborhoods or price ranges).

We aim to reduce the dataset to 100 rows or fewer by selecting only a few rows from each neighborhood and excluding rows with grouped apartment information.


In [180]:
# Select only the relevant columns for analysis
df = df[['title', 'price', 'surface', 'rooms', 'bathrooms', 'typology', 'macrozone', 'microzone', 'latitude', 'longitude']]

# Filter to ensure each row represents a single apartment
# Here, we're excluding rows where grouped apartment information may be present
df = df[df['title'].str.contains('appartamenti', case=False) == False]

# Sample data to keep rows under 100 and maintain a balance across neighborhoods
# Ensure there are around 10 samples per macrozone but respect the total of 100 rows
max_samples_per_macrozone = min(10, len(df) // df['macrozone'].nunique())
df = df.groupby('macrozone', group_keys=False).apply(lambda x: x.sample(min(max_samples_per_macrozone, len(x)))).reset_index(drop=True)

# If the total number of rows exceeds 100, sample again
if len(df) > 100:
    df = df.sample(n=100, random_state=42).reset_index(drop=True)

# Check the final shape of the DataFrame
df.shape


C:\Users\nicol\AppData\Local\Temp\ipykernel_21376\1120672493.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('macrozone', group_keys=False).apply(lambda x: x.sample(min(max_samples_per_macrozone, len(x)))).reset_index(drop=True)


(100, 10)

## Step 5: Final Data Review

Now that the data has been cleaned and filtered, we can proceed with a final review to ensure that it meets the project requirements.


In [185]:
# Final review of the data
df.info()

# Display the first few rows of the cleaned dataset
df.sample(10)

<class 'pandas.core.frame.DataFrame'>
Index: 99 entries, 0 to 99
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      99 non-null     object 
 1   price      99 non-null     int64  
 2   surface    99 non-null     int64  
 3   rooms      99 non-null     int32  
 4   bathrooms  99 non-null     int32  
 5   typology   99 non-null     object 
 6   macrozone  99 non-null     object 
 7   microzone  99 non-null     object 
 8   latitude   99 non-null     float64
 9   longitude  99 non-null     float64
dtypes: float64(2), int32(2), int64(2), object(4)
memory usage: 7.7+ KB


,title,price,surface,rooms,bathrooms,typology,macrozone,microzone,latitude,longitude
20,"Trilocale via Piero della Francesca 34, Sempio...",595000,97,3,2,Appartamento,"Fiera, Sempione, City Life, Portello",Sempione,45.4838,9.1643
68,"Trilocale via Giovanni Battista Sammartini 49,...",460000,101,3,1,Appartamento,"Maggiolina, Istria",Cascina dei Pomi,45.4922,9.2086
27,"Trilocale via del Torchio 4, Carrobbio, Milano",795000,85,3,1,Appartamento,Centro,Carrobbio,45.4603,9.1808
36,"Trilocale via della Torre 15, Turro, Milano",450000,82,3,1,Appartamento,"Precotto, Turro",Turro,45.5008,9.2236
69,"Quadrilocale via sismondi 6, Plebisciti - Susa...",980000,160,4,2,Appartamento,"Città Studi, Susa",Plebisciti - Susa,45.4655,9.2248
97,"Bilocale via Michelangelo Buonarroti 4, Vercel...",589000,77,2,2,Appartamento,"Solari, Washington",Vercelli - Wagner,45.4674,9.1555
52,"Trilocale via Luigi Settembrini 43, Centrale, ...",590000,110,3,2,Appartamento,"Centrale, Repubblica",Centrale,45.4842,9.2084
62,"Trilocale via degli Imbriani, Bovisa, Milano",470000,96,3,1,Appartamento,"Affori, Bovisa",Bovisa,45.4996,9.1687
17,"Attico via Gallarate 55, Certosa, Milano",699000,203,4,2,Attico,"Viale Certosa, Cascina Merlata",Certosa,45.4952,9.1369
67,"Trilocale viale Berengario 11, Monte Rosa - Lo...",765000,99,3,2,Appartamento,"Fiera, Sempione, City Life, Portello",Monte Rosa - Lotto,45.4754,9.1512


In [186]:
df.to_csv("dataset.csv")